## Basic dependencies
(some may not be used)

In [3]:
import os
import pickle
import random
import time
import gdown
import tarfile
import numpy as np
import PIL
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, concatenate, Flatten, Lambda, Concatenate
from keras.optimizers import Adam
from matplotlib import pyplot as plt

## Data

### Load ids from class_info.pickle file

In [4]:
def load_ids(class_info_file_path):
    data = open(class_info_file_path).read().replace('\r\n', '\n')
    dst = class_info_file_path + ".tmp"
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids

### Load Embeddings

In [5]:
def load_embeddings(embeddings_file_path):
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings
 

### Load names.pickle and return a list of all names

In [6]:
def load_filenames(filenames_file_path):
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

### Load bounding boxes from CUB_200_211 file

In [7]:
def load_bounding_boxes(dataset_dir):
    # Paths
    bounding_boxes_path = os.path.join(dataset_dir, 'bounding_boxes.txt')
    file_paths_path = os.path.join(dataset_dir, 'images.txt')

    # Read bounding_boxes.txt and images.txt file
    df_bounding_boxes = pd.read_csv(bounding_boxes_path, sep='\s+', header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, sep='\s+', header=None)

    # Create a list of file names
    file_names = df_file_names[1].tolist()

    # Create a dictionary of file_names and bounding boxes
    filename_boundingbox_dict = {img_file[:-4]: [] for img_file in file_names[:2]}

    # Assign a bounding box to the corresponding image
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][1:].tolist()
        key = file_names[i][:-4]
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/var/folders/lc/4x9mrwg96lbdys8fhkp84l0c0000gn/T/ipykernel_15670/4108616333.py:7: SyntaxWarning: invalid escape sequence '\s'
  df_bounding_boxes = pd.read_csv(bounding_boxes_path, sep='\s+', header=None).astype(int)
/var/folders/lc/4x9mrwg96lbdys8fhkp84l0c0000gn/T/ipykernel_15670/4108616333.py:8: SyntaxWarning: invalid escape sequence '\s'
  df_file_names = pd.read_csv(file_paths_path, sep='\s+', header=None)


### Load and resize images

In [8]:
def get_img(img_path, bbox, image_size):
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img

### Load Dataset

In [9]:
def load_dataset(filenames_file_path, class_info_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    filenames = load_filenames(filenames_file_path)
    class_ids = load_ids(class_info_file_path)
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    for index in range(min(len(filenames), len(all_embeddings))):
        filename = filenames[index]
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{}/images/{}.jpg'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)

            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            embedding = all_embeddings1[embedding_ix, :]

            X.append(np.array(img))
            y.append(class_ids[index])
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, y, embeddings

In [10]:
if not(os.path.isdir("CUB_200_2011/")):
        gdown.download("https://drive.google.com/uc?export=download&id=1SaagQ8Yr2PC5b45E7WHu3q2dWLXj9c_G", 'CUB_200_2011.tgz', quiet=False)
        print(os.listdir())
        tar = tarfile.open('CUB_200_2011.tgz', 'r:gz')
        tar.extractall()
        tar.close()
if not(os.path.exists("dataset/test/embeddings.pickle")):
    gdown.download("https://drive.google.com/uc?export=download&id=1P8C8MCF_wQLcl6bVl3RwgAVmVtRDyymH", 'birds/test/embeddings.pickle', quiet=False)
if not(os.path.exists("dataset/train/embeddings.pickle")):
    gdown.download("https://drive.google.com/uc?export=download&id=1P8C8MCF_wQLcl6bVl3RwgAVmVtRDyymH", 'birds/train/embeddings.pickle', quiet=False)

In [11]:
data_dir = "dataset/"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"

In [12]:
embeddings_file_path_train = train_dir + "/embeddings.pickle"
embeddings_file_path_test = test_dir + "/embeddings.pickle"

In [13]:
filenames_file_path_train = train_dir + "/names.pickle"
filenames_file_path_test = test_dir + "/names.pickle"

class_info_file_path_train = train_dir + "/class_info.pickle"
class_info_file_path_test = test_dir + "/class_info.pickle"

In [14]:
filenames_file_path_train = train_dir + "/names.pickle"
filenames_file_path_test = test_dir + "/names.pickle"

class_info_file_path_train = train_dir + "/class_info.pickle"
class_info_file_path_test = test_dir + "/class_info.pickle"

In [15]:
cub_dataset_dir = "CUB_200_2011/CUB_200_2011"

In [16]:
""""
Load datasets
"""
X_train, y_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                    class_info_file_path=class_info_file_path_train,
                                                    cub_dataset_dir=cub_dataset_dir,
                                                    embeddings_file_path=embeddings_file_path_train,
                                                    image_size=(64, 64))

X_test, y_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                class_info_file_path=class_info_file_path_test,
                                                cub_dataset_dir=cub_dataset_dir,
                                                embeddings_file_path=embeddings_file_path_test,
                                                image_size=(64, 64))


embeddings:  (2933, 10, 1024)
Embeddings shape: (2933, 10, 1024)
embeddings:  (2933, 10, 1024)
Embeddings shape: (2933, 10, 1024)


## GAN

In [17]:
image_size = 64
batch_size = 64
z_dim = 100
stage1_generator_lr = 0.0002
stage1_discriminator_lr = 0.0002
stage1_lr_decay_step = 600
epochs = 1000
condition_dim = 128
embedding_dim = 1024

### Function to generate conditioning vector

In [18]:
def generate_c(x):
    mean = x[:, :condition_dim]
    log_sigma = x[:, condition_dim:]
    stddev = tf.exp(log_sigma)
    epsilon = tf.random.normal(shape=(mean.shape[1],))
    c = stddev * epsilon + mean
    return c

In [19]:
# Build the generator model
def build_generator():
    input_layer = Input(shape=(embedding_dim,))
    x = Dense(condition_dim * 2)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)
    c = Lambda(generate_c)(mean_logsigma)

    noise_input = Input(shape=(z_dim,))
    gen_input = Concatenate()([c, noise_input])

    x = Dense(128 * 8 * 4 * 4, use_bias=False)(gen_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((4, 4, 128 * 8))(x)

    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    output = Activation('tanh')(x)

    model = Model([input_layer, noise_input], output)
    return model

In [20]:
# Build the discriminator model
def build_discriminator():
    img_input = Input(shape=(image_size, image_size, 3))

    x = tf.keras.layers.Conv2D(64, kernel_size=4, strides=2, padding='same', use_bias=False)(img_input)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.Conv2D(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.Conv2D(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(img_input, x)
    return model

In [21]:
# Build the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False

    embedding_input = Input(shape=(embedding_dim,))
    noise_input = Input(shape=(z_dim,))

    img = generator([embedding_input, noise_input])
    validity = discriminator(img)

    model = Model([embedding_input, noise_input], validity)
    return model


In [22]:
# Compile models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Training
real_labels = np.ones((batch_size, 1)) * 0.9
fake_labels = np.zeros((batch_size, 1)) * 0.1

# Assuming embeddings_train and X_train are your training data
embeddings_train = np.random.randn(2933, 10, 1024)  # Placeholder
X_train = np.random.randn(2933, image_size, image_size, 3)  # Placeholder

tensorboard = TensorBoard(log_dir="logs/")
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)

/Users/devanshurana/Desktop/Projects/text_2_image-gan/.venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [23]:
%reload_ext autoreload
%autoreload 2

: 

In [24]:
for epoch in range(epochs):
    print("<------------------------------------------------------------------------------>")
    print(f"Epoch: {epoch}")

    gen_losses = []
    dis_losses = []

    number_of_batches = int(X_train.shape[0] / batch_size)
    for batch in range(number_of_batches):
        print(f"Batch: {batch + 1}/{number_of_batches}")

        # Train discriminator
        z_noise = np.random.normal(0, 1, (batch_size, z_dim))
        real_imgs = X_train[batch * batch_size:(batch + 1) * batch_size]
        embedding_batch = embeddings_train[batch * batch_size:(batch + 1) * batch_size]

        fake_imgs = generator.predict([embedding_batch[:, 0, :], z_noise])

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        dis_losses.append(d_loss)

        # Train generator
        g_loss = gan.train_on_batch([embedding_batch[:, 0, :], z_noise], real_labels)
        gen_losses.append(g_loss)

    print(f"Discriminator Loss: {np.mean(dis_losses)}, Generator Loss: {np.mean(gen_losses)}")

    # Log losses
    tensorboard.on_epoch_end(epoch, {'discriminator_loss': np.mean(dis_losses), 'generator_loss': np.mean(gen_losses)})

    # Generate and save images every 10 epochs
    if epoch % 10 == 0:
        z_noise = np.random.normal(0, 1, (batch_size, z_dim))
        embedding_batch = embeddings_train[:batch_size]
        generated_imgs = generator.predict([embedding_batch[:, 0, :], z_noise])

        for i, img in enumerate(generated_imgs[:5]):
            plt.imshow((img + 1) / 2)
            plt.axis('off')
            plt.savefig(f'results/gen_{epoch}_{i}.png')
            plt.close()

Epoch: 0
Batch: 1/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step


/Users/devanshurana/Desktop/Projects/text_2_image-gan/.venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Batch: 2/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
Batch: 3/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
Batch: 4/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 282ms/step
Batch: 5/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
Batch: 6/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step
Batch: 7/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
Batch: 8/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step
Batch: 9/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 280ms/step
Batch: 10/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 322ms/step
Batch: 11/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step
Batch: 12/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step
Batch: 13/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 409ms/step
Batch: 14/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 327ms/step
Batch: 15/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 331ms/step
Batch: 16/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step
Batch: 17/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 331ms/step
Batch: 18/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step
Batch: 19/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 391ms/step
Batch: 20/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 373ms/step
Batch: 21/45
2/2 ━━━

In [ ]:
generator.save("generator.h5")
discriminator.save("discriminator.h5")

In [ ]:
# Save models
stage1_gen.save_weights("generator.h5")
stage1_dis.save_weights("discriminator.h5")